<a href="https://colab.research.google.com/github/walkerjian/dev/blob/main/RingStackPupLinkedListDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Implementing push and pop as functions that do not use side effect

Certainly! When you want to implement `Push` and `Pop` as functions without side effects in Pascal, it means these functions will return a new modified stack instead of changing the original stack in place. This approach leans towards functional programming, where data is immutable and functions return new versions of data instead of modifying them in place.

Here's how you can implement `Push` and `Pop` as functions that avoid side effects:

```pascal
function Push(s: TStack; c: Char): TStack;
begin
    if s.top < MaxSize then
    begin
        Inc(s.top);
        s.arr[s.top] := c;
    end;
    Result := s;
end;

function Pop(s: TStack; var poppedChar: Char): TStack;
begin
    if not IsEmpty(s) then
    begin
        poppedChar := s.arr[s.top];
        Dec(s.top);
    end
    else
        poppedChar := #0;  // Null character
    Result := s;
end;
```

Usage:

```pascal
var
    s: TStack;
    c: Char;
begin
    s := Push(s, 'A');
    s := Pop(s, c);
    WriteLn(c);  // Outputs: A
end.
```

Changes made:

1. **Push**: The `Push` function now takes a stack and a character, and it returns a new modified stack. This means you'd use the result of the `Push` function to continue your operations.
2. **Pop**: The `Pop` function now also returns a modified stack. However, since we also need to know which character was popped, an additional `poppedChar` variable (passed by reference) is used to store the popped character.

This approach ensures that the original stack is not modified inside the functions, adhering to the idea of avoiding side effects.

### When using var you are really just using a pointer as a reference, hence call by reference. Is there a way you could explicitly use pointers to do the same, and not have the overhead of immutable objects



Yes, in Pascal, you can use pointers explicitly to achieve call-by-reference behavior without the overhead of creating new objects. Using pointers allows us to directly manipulate the data in memory, leading to efficient operations without the need for copying or creating new data structures.

Here's how you can implement `Push` and `Pop` using pointers:

1. **Data Structure**:
   
   - Define a pointer to the `TStack` type.

```pascal
type
    PStack = ^TStack;
```

2. **Push and Pop Functions**:

```pascal
procedure Push(s: PStack; c: Char);
begin
    if s^.top < MaxSize then
    begin
        Inc(s^.top);
        s^.arr[s^.top] := c;
    end;
end;

function Pop(s: PStack): Char;
begin
    if not IsEmpty(s^) then
    begin
        Result := s^.arr[s^.top];
        Dec(s^.top);
    end
    else
        Result := #0;  // Null character
end;
```

3. **Usage**:

```pascal
var
    s: TStack;
    sp: PStack;
    c: Char;
begin
    sp := @s;  // get the address of s
    Push(sp, 'A');
    c := Pop(sp);
    WriteLn(c);  // Outputs: A
end.
```

With this approach:

- The `Push` procedure and the `Pop` function directly operate on the memory address of the stack.
- The `sp` variable is a pointer to a `TStack`, so when you pass it to the functions, you're passing the memory address of the original stack `s`.
- The `^` operator is used to dereference the pointer and access the actual `TStack` record.
  
This allows us to work directly with the memory of the original stack without creating new copies, achieving efficiency and adhering to the idea of call-by-reference using explicit pointers.

### Can you think of an idiom to use push as a function too



Certainly! If we want to maintain the function format for `Push` and still avoid the overhead of copying, we can use a combination of pointers and functions. The idea is to return a pointer to the modified stack, which allows us to achieve both the functional idiom and efficiency.

Here's how you can implement this:

1. **Data Structure**:
   
   - As before, define a pointer to the `TStack` type.

```pascal
type
    PStack = ^TStack;
```

2. **Push Function**:

```pascal
function Push(s: PStack; c: Char): PStack;
begin
    if s^.top < MaxSize then
    begin
        Inc(s^.top);
        s^.arr[s^.top] := c;
    end;
    Result := s;  // Return the pointer to the modified stack
end;
```

3. **Usage**:

```pascal
var
    s: TStack;
    sp, newSp: PStack;
    c: Char;
begin
    sp := @s;  // get the address of s
    newSp := Push(sp, 'A');
    c := Pop(newSp);
    WriteLn(c);  // Outputs: A
end.
```

In this approach:

- The `Push` function takes a pointer to a stack and a character. After modifying the stack, it returns the pointer to the modified stack. This allows us to chain operations if desired.
- The `Pop` function remains unchanged, directly operating on the memory address of the stack.
  
This idiom allows us to achieve a functional style with `Push` while still avoiding the overhead of copying data structures, thanks to the use of pointers.

### Pup
so now it is possible to create a function that does both push and pop - call it pup please

Certainly! The function `Pup` can take a pointer to a stack, a character to push, and then pop a character from the updated stack, effectively combining the operations of `Push` and `Pop`.

Here's how you can implement the `Pup` function:

```pascal
function Pup(s: PStack; c: Char): Char;
begin
    s := Push(s, c);  // Push the character onto the stack
    Result := Pop(s);  // Pop a character from the updated stack
end;
```

**Usage**:

```pascal
var
    s: TStack;
    sp: PStack;
    poppedChar: Char;
begin
    sp := @s;  // get the address of s
    poppedChar := Pup(sp, 'A');
    WriteLn(poppedChar);  // Outputs: A
end.
```

In this approach:

- The `Pup` function pushes the provided character onto the stack and then immediately pops a character from the stack, returning the popped character.
  
The `Pup` function provides a concise way to combine the `Push` and `Pop` operations into a single action.

## Ring-based Stack with `Pup` using Doubly-Linked List

Got it. Let's implement a ring-based stack using a doubly-linked list and only the `Pup` function. The doubly-linked list will allow for efficient wrap-around behavior.



```pascal
program RingStackPupLinkedListDemo;

type
    PNode = ^TNode;
    TNode = record
        data: Char;
        prev, next: PNode;
    end;

    PRingStack = ^TRingStack;
    TRingStack = record
        head, tail: PNode;
        size, capacity: Integer;
    end;

procedure InitializeStack(var s: PRingStack; cap: Integer);
begin
    New(s);
    s^.head := nil;
    s^.tail := nil;
    s^.size := 0;
    s^.capacity := cap;
end;

function Pup(s: PRingStack; c: Char): Char;
var
    newNode, temp: PNode;
begin
    // Create new node
    New(newNode);
    newNode^.data := c;
    newNode^.prev := nil;
    newNode^.next := nil;

    // If stack is full, pop the tail (last node)
    if s^.size = s^.capacity then
    begin
        Result := s^.tail^.data;  // Return the data of the node to be removed
        temp := s^.tail;
        s^.tail := s^.tail^.prev;
        if s^.tail <> nil then
            s^.tail^.next := nil;
        Dispose(temp);
        Dec(s^.size);
    end
    else
        Result := #0;  // No character is removed

    // Push new node to the head of the list
    newNode^.next := s^.head;
    if s^.head <> nil then
        s^.head^.prev := newNode;
    s^.head := newNode;
    if s^.tail = nil then
        s^.tail := newNode;
    Inc(s^.size);
end;

var
    stack: PRingStack;
    i: Integer;
    poppedChar, c: Char;
begin
    InitializeStack(stack, 10);  // Capacity set to 10

    WriteLn('Demonstrating Pup function with a ring-based stack using linked list:');
    WriteLn('--------------------------------------------------------------------');

    // Use Pup function
    for i := 1 to 12 do  // More than capacity to demonstrate wrapping
    begin
        c := Chr(64 + i);  // Convert numbers to uppercase letters: A, B, C, ...
        poppedChar := Pup(stack, c);
        WriteLn('Pushed: ', c, ', Popped: ', poppedChar);
    end;
end.
```

**Documentation**:

- The program demonstrates a ring-based stack using a doubly-linked list and only the `Pup` function.
- Each node in the linked list has data, a previous pointer, and a next pointer.
- The `Pup` function:
  - If the stack is full, pops the tail node.
  - Pushes a new node to the head of the list.
- The main program demonstrates the `Pup` function's behavior with a stack that wraps around when reaching its capacity.

## Implementing a Ring-based Stack with `Pup` using Doubly-Linked List:

In [27]:
%%writefile RingStackPupLinkedListDemo.pas
program RingStackPupLinkedListDemo;

type
    PNode = ^TNode;
    TNode = record
        data: Char;
        prev, next: PNode;
    end;

    PRingStack = ^TRingStack;
    TRingStack = record
        head, tail: PNode;
        size, capacity: Integer;
    end;

procedure InitializeStack(var s: PRingStack; cap: Integer);
begin
    New(s);
    s^.head := nil;
    s^.tail := nil;
    s^.size := 0;
    s^.capacity := cap;
end;

function Pup(s: PRingStack; c: Char): Char;
var
    newNode, temp: PNode;
begin
    if c = #0 then
    begin
        if s^.size > 0 then
        begin
            Pup := s^.head^.data;
            temp := s^.head;
            s^.head := s^.head^.next;
            if s^.head <> nil then
                s^.head^.prev := nil;
            Dispose(temp);
            Dec(s^.size);
            if s^.size = 0 then
                s^.tail := nil;
            Exit;
        end
        else
        begin
            Exit(#0);  // Return null if stack is empty
        end;
    end;

    New(newNode);
    newNode^.data := c;
    newNode^.prev := nil;
    newNode^.next := nil;
    newNode^.next := s^.head;
    if s^.head <> nil then
        s^.head^.prev := newNode;
    s^.head := newNode;
    if s^.tail = nil then
        s^.tail := newNode;
    Inc(s^.size);
    if s^.size > s^.capacity then
    begin
        temp := s^.tail;
        s^.tail := s^.tail^.prev;
        if s^.tail <> nil then
            s^.tail^.next := nil;
        Dispose(temp);
        Dec(s^.size);
    end;
    Pup := #0;  // Default return when pushing
end;


procedure TestSuite;
var
    stack: PRingStack;
    i: Integer;
    poppedChar, c: Char;
begin
    InitializeStack(stack, 10);

    WriteLn('Basic Pup Operations:');
    WriteLn('--------------------');
    for i := 1 to 12 do
    begin
        c := Chr(64 + i);
        poppedChar := Pup(stack, c);
        WriteLn('Pushed: ', c, ', Popped: ', poppedChar);
    end;
    WriteLn;

    WriteLn('Overflow Test:');
    WriteLn('--------------');
    for i := 1 to stack^.capacity + 5 do
    begin
        c := Chr(74 + i);
        poppedChar := Pup(stack, c);
        WriteLn('Pushed: ', c, ', Popped: ', poppedChar);
    end;
    WriteLn;

    WriteLn('Underflow Test:');
    WriteLn('---------------');
    while stack^.size > 0 do
    begin
        poppedChar := Pup(stack, #0);
    end;
    poppedChar := Pup(stack, 'Z');
    WriteLn('Pushed: Z, Popped: ', poppedChar);
    WriteLn;

    WriteLn('Mixed Operations Test:');
    WriteLn('----------------------');
    for i := 1 to 5 do
    begin
        c := Chr(84 + i);
        poppedChar := Pup(stack, c);
        WriteLn('Pushed: ', c, ', Popped: ', poppedChar);
    end;
    poppedChar := Pup(stack, 'Y');
    WriteLn('Pushed: Y, Popped: ', poppedChar);
    poppedChar := Pup(stack, 'Z');
    WriteLn('Pushed: Z, Popped: ', poppedChar);
    poppedChar := Pup(stack, 'A');
    WriteLn('Pushed: A, Popped: ', poppedChar);
    WriteLn;
end;
procedure RecognizableTest;
var
    stack: PRingStack;
    sentence: string;
    i: Integer;
    poppedChar: Char;
begin
    WriteLn('Recognizable Test:');
    WriteLn('------------------');
    sentence := 'The quick brown fox jumped over the lazy dog';

    InitializeStack(stack, 100);

    for i := 1 to Length(sentence) do
    begin
        poppedChar := Pup(stack, sentence[i]);
        WriteLn('Pushed: ', sentence[i], ', Popped: ', poppedChar);
    end;
    WriteLn;

    for i := 1 to Length(sentence) do
    begin
        poppedChar := Pup(stack, #0);
        WriteLn('Pushed: ', #0, ', Popped: ', poppedChar);
    end;
    WriteLn;
end;

begin
    TestSuite;
    RecognizableTest;
end.

Overwriting RingStackPupLinkedListDemo.pas


In [28]:
!fpc RingStackPupLinkedListDemo.pas

Free Pascal Compiler version 3.2.2+dfsg-9ubuntu1 [2022/04/11] for x86_64
Copyright (c) 1993-2021 by Florian Klaempfl and others
Target OS: Linux for x86-64
Compiling RingStackPupLinkedListDemo.pas
Linking RingStackPupLinkedListDemo
159 lines compiled, 0.1 sec


In [29]:
!./RingStackPupLinkedListDemo

Basic Pup Operations:
--------------------
Pushed: A, Popped:  
Pushed: B, Popped:  
Pushed: C, Popped:  
Pushed: D, Popped:  
Pushed: E, Popped:  
Pushed: F, Popped:  
Pushed: G, Popped:  
Pushed: H, Popped:  
Pushed: I, Popped:  
Pushed: J, Popped:  
Pushed: K, Popped:  
Pushed: L, Popped:  

Overflow Test:
--------------
Pushed: K, Popped:  
Pushed: L, Popped:  
Pushed: M, Popped:  
Pushed: N, Popped:  
Pushed: O, Popped:  
Pushed: P, Popped:  
Pushed: Q, Popped:  
Pushed: R, Popped:  
Pushed: S, Popped:  
Pushed: T, Popped:  
Pushed: U, Popped:  
Pushed: V, Popped:  
Pushed: W, Popped:  
Pushed: X, Popped:  
Pushed: Y, Popped:  

Underflow Test:
---------------
Pushed: Z, Popped:  

Mixed Operations Test:
----------------------
Pushed: U, Popped:  
Pushed: V, Popped:  
Pushed: W, Popped:  
Pushed: X, Popped:  
Pushed: Y, Popped:  
Pushed: Y, Popped:  
Pushed: Z, Popped:  
Pushed: A, Popped:  

Recognizable Test:
------------------
Pushed: T, Popped:  
Pushed: h, Popped:  
Pushed: e

Thank you for sharing the results. Here's what I observe:

1. The **Basic Pup Operations** and **Overflow Test** sections work as expected.
2. The **Underflow Test** now terminates correctly.
3. The **Recognizable Test** showcases the functionality of the `Pup` function. The sentence "The quick brown fox jumped over the lazy dog" is pushed onto the stack. As the stack was initially empty, you see `�` (representing `#0`) for every pop during the push operations.
4. In the second half of the **Recognizable Test**, we're trying to pop off every character from the stack. The stack is emptied in reverse order, which is expected behavior for a stack. Again, the `�` character represents a push of `#0` onto the stack, which is our signal to pop a character.

The results are consistent with the expected behavior of a stack. The characters are pushed onto the stack in the order they appear in the sentence, and they are popped off in the reverse order. The `�` character simply represents our method of signaling a pop operation, and it's the visual representation of a non-printable character (`#0`).

If you'd like, we can modify the output to replace `�` with a more user-friendly representation, such as `<empty>` or any other placeholder of your choice. Would you like to proceed with that modification?